# GFW climate biomass widgets

In [23]:
import pandas as pd
import geopandas as gpd
import numpy as np
import matplotlib.pyplot as plt
from tqdm import tqdm
import requests
from pprint import pprint
import h5py

## Table with biomass density and total biomass

**GADM 3.6 admin 2**

In [5]:
df = gpd.read_file('/Users/ikersanchez/Vizzuality/DATA/gadm36_shp/gadm36.shp')

In [6]:
df = df[['GID_0', 'ID_0', 'NAME_0', 'ID_1', 'NAME_1', 'ID_2', 'NAME_2']]

In [7]:
df.head()

,GID_0,ID_0,NAME_0,ID_1,NAME_1,ID_2,NAME_2
0,AFG,1,Afghanistan,1,Badakhshan,1,Baharak
1,AFG,1,Afghanistan,1,Badakhshan,2,Darwaz
2,AFG,1,Afghanistan,1,Badakhshan,3,Fayzabad
3,AFG,1,Afghanistan,1,Badakhshan,4,Ishkashim
4,AFG,1,Afghanistan,1,Badakhshan,5,Jurm


In [8]:
df['NAME_2'].replace('', np.nan, inplace=True)

In [9]:
df.dropna(subset=['NAME_2'], inplace=True)

In [10]:
df.drop_duplicates(subset=['NAME_0', 'NAME_1', 'NAME_2'], keep='first', inplace=True)

The API contains an endpoint for `whrc-biomass` to compute the total biomass and biomass density of a given municipality.

In [29]:
url = "https://production-api.globalforestwatch.org/v1/"
r = requests.get(url + 'whrc-biomass/admin/ZWE/6/31')
print(f'Status code: {r.status_code}')
pprint(r.json())

Status code: 200
{'data': {'attributes': {'areaHa': 611557.6528581444,
                         'biomassDensity': 3.9051680590532816,
                         'totalBiomass': 2388235.4122112202},
          'id': None,
          'type': 'whrc-biomass'}}


In [ ]:
url = "https://production-api.globalforestwatch.org/v1/"

nValues = len(df)

with h5py.File('./data/biomass.hdf5', 'w') as f:
    density = f.create_dataset("density", shape=(nValues,), dtype=np.float32)
    total = f.create_dataset("total", shape=(nValues,), dtype=np.float32)
    
    for i in tqdm(range(nValues)):
        for attempt in range(4):
            try:
                r = requests.get(url + f"whrc-biomass/admin/{df['GID_0'].iloc[i+325]}/{df['ID_1'].iloc[i+325]}/{df['ID_2'].iloc[i+325]}")
                density[i] = r.json().get('data').get('attributes').get('biomassDensity')
                total[i] = r.json().get('data').get('attributes').get('totalBiomass') 
            except:
                if r.status_code != 404:
                    continue
            else:
                break

In [4]:
with h5py.File('./data/biomass.hdf5', 'r') as f:
    density = f['density'][:]
    total = f['total'][:]
    
    print(len(total))

45958


In [15]:
df['biomassDensity'] = density
df['totalBiomass'] = total

Save table

In [22]:
df.to_csv("./data/whrc-biomass.csv")